# Refference
most relative match is jane street also have anonymized features 

link: https://www.kaggle.com/c/jane-street-market-prediction/discussion/205112

but jane street predict target is known rather than this 

1. chris's transformer https://www.kaggle.com/cdeotte/tensorflow-transformer-0-112
2. Toda origin notebook https://www.kaggle.com/takamichitoda/ump-train-transformer-on-tpu
3. CV strategy https://www.kaggle.com/c/ubiquant-market-prediction/discussion/302429
4. out of mean abnormal data 
    https://www.kaggle.com/c/ubiquant-market-prediction/discussion/305067   < 250 | > 250 < 500 
    https://www.kaggle.com/lucasmorin/umap-data-analysis-applications/
5. PurgedTimeSeriesCV https://www.kaggle.com/gogo827jz/jane-street-ffill-xgboost-purgedtimeseriescv

# Version 
1. version 1 copy toda's notebook
2. version 2 test pcc loss  reffer: pcc is trick https://www.kaggle.com/c/ubiquant-market-prediction/discussion/306322
3. version 5 PurgedGroupTimeSeriesSplit

# TODO 
1. not cv split 
2. not time serices features
3. not feature auto select

# Piplines links
1. data half precise to pickle https://www.kaggle.com/drzhuzhe/ubiquant-market-prediction-eda
2. data to npy https://www.kaggle.com/drzhuzhe/ump-data-preprocess

In [ ]:
import os
import gc
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt


from sklearn.metrics import mean_squared_error
#from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import TimeSeriesSplit

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K

In [ ]:
class GCF:
    INPUT_ROOT = "/kaggle/input/ump-data-preprocess/"
    SEED = 0
    
    N_EPOCHS = 50
    BATCH_SIZE = 4096
    #BATCH_SIZE = 1024
    EARLY_STOPPING_PATIENCE = 10
    EARLY_STOPPING_MIN_DELTA = 1e-3
    ALL_TRAIN_ADD_EPOCH = 3
    
    # Transformer Parameters
    EMBED_DIM = 64//2
    N_HEAD = 8
    FF_DIM = 128//2
    DROPOUT = 0.0
    N_BLOCK = 4

In [ ]:
%%time
# use std features
X = np.load(f"{GCF.INPUT_ROOT}/features_std_scaled.npy")
y = np.load(f"{GCF.INPUT_ROOT}/targets.npy")
time_id = np.load(f"{GCF.INPUT_ROOT}/time_id.npy")

In [ ]:
# wht > 700 ?
#is_train = np.where((time_id <= 1066) & (time_id > 700), True, False)
#is_test = time_id > 1066

#sum(is_train), sum(is_test)

In [ ]:
# this is code slightly modified from the sklearn docs here:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    
    cmap_cv = plt.cm.coolwarm

    jet = plt.cm.get_cmap('jet', 256)
    seq = np.linspace(0, 1, 256)
    _ = np.random.shuffle(seq)   # inplace
    cmap_data = ListedColormap(jet(seq))

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['target', 'day']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

In [ ]:
n_splits = 5
#group_gap = 31
group_gap = 100

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
from matplotlib.colors import ListedColormap

In [ ]:
# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
%%time

fig, ax = plt.subplots()
gkf = PurgedGroupTimeSeriesSplit(n_splits = n_splits, group_gap = group_gap)
plot_cv_indices(gkf, X, y, time_id, ax, 5, lw = 20)
plt.show()

In [ ]:
# wht > 700 ?
#is_train = np.where((time_id <= 1066) & (time_id > 700), True, False)
#is_test = time_id > 1066

#sum(is_train), sum(is_test)

In [ ]:
#for fold, (tr, te) in enumerate(gkf.split(X, y, time_id)):
#    print("train len", len(X[tr]), "test len", len(X[te]))
#    X_tr, X_val = X[tr], X[te]
#    y_tr, y_val = y[tr], y[te]
#    is_train = tr
#    is_test = te
#    break

# model

In [ ]:
TPU = False
if TPU == True:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    print('Running on TPU ', tpu.master())
    print("REPLICAS: ", strategy.num_replicas_in_sync)
else:
    # USE MULTIPLE GPUS
    if os.environ["NVIDIA_VISIBLE_DEVICES"].count(',') == 0:
        strategy = tf.distribute.get_strategy()
        print('single strategy')
    else:
        strategy = tf.distribute.MirroredStrategy()
        print('multiple strategy')

In [ ]:
# USE MIXED PRECISION
# UNFORTUNATELY FOR THIS MODEL, MIXED PRECISION HURTS MODEL PERFORMANCE
#tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
#print('Mixed precision enabled')

In [ ]:
# https://www.kaggle.com/c/ubiquant-market-prediction/discussion/302977

def correlationMetric(x, y, axis=-2):
    """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xvar = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    yvar = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xvar * yvar)
    return tf.constant(1.0, dtype=x.dtype) - corr


def correlationLoss(x,y, axis=-2):
    """Loss function that maximizes the pearson correlation coefficient between the predicted values and the labels,
    while trying to have the same mean and variance"""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xsqsum = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    ysqsum = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xsqsum * ysqsum)
    sqdif = tf.reduce_sum(tf.math.squared_difference(x, y), axis=axis) / n / tf.sqrt(ysqsum / n)
    return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr + (0.01 * sqdif)) , dtype=tf.float32 )


#　https://www.kaggle.com/c/ubiquant-market-prediction/discussion/301987
def pearson_coef(data):
    return data.corr()['target']['preds']

def comp_metric(time_id, y, pred):
    return np.mean(
        pd.DataFrame(np.stack([time_id, y, pred]).T, columns=['time_id', 'target', 'preds']
    ).groupby('time_id').apply(pearson_coef))

In [ ]:
feat_dim = X.shape[-1]
print(feat_dim)

In [ ]:
# https://www.kaggle.com/pratikskarnik/riiid-keras-transformer-starter
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8, **kwargs):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output
    
    def get_config(self):
        config = {
            "embed_dim" : self.embed_dim,
            "num_heads" : self.num_heads,
            "projection_dim" : self.projection_dim,
            "query_dense" : self.query_dense,
            "key_dense" : self.key_dense,
            "value_dense" : self.value_dense,
            "combine_heads" : self.combine_heads,
        }
        base_config = super(MultiHeadSelfAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))  


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim=GCF.EMBED_DIM, feat_dim=feat_dim, num_heads=GCF.N_HEAD, ff_dim=GCF.FF_DIM, rate=GCF.DROPOUT, **kwargs):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(num_heads=num_heads, embed_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="gelu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        config = {
            "att" : self.att,
            "ffn" : self.ffn,
            "layernorm1" : self.layernorm1,
            "layernorm2" : self.layernorm2,
            "dropout1" : self.dropout1,
            "dropout2" : self.dropout2,
        }
        base_config = super(TransformerBlock, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
# https://www.kaggle.com/cdeotte/tensorflow-transformer-0-112

def create_model():
    inputs = layers.Input(shape=(1, feat_dim))
    
    # "EMBEDDING LAYER"
    x = layers.Dense(GCF.EMBED_DIM)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    
    # TRANSFORMER BLOCKS
    for k in range(GCF.N_BLOCK):
        #x_old = x
        transformer_block = TransformerBlock(GCF.EMBED_DIM, feat_dim, GCF.N_HEAD, GCF.FF_DIM, GCF.DROPOUT)
        x = transformer_block(x)
        #x = 0.7*x + 0.3*x_old # SKIP CONNECTION
    
    x = layers.GlobalAveragePooling1D()(x)
    #x = layers.Dropout(0.2)(x)
    x = layers.Dense(20, activation="relu")(x)
    #x = layers.Dropout(0.2)(x)
    
    # REGRESSION HEAD
    outputs = layers.Dense(1, activation="linear")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=tf.optimizers.Adam(1e-4),
        # version 1 
        loss='mse',
        # version 2 
        #loss=correlationLoss,
        metrics=[keras.metrics.RootMeanSquaredError(), correlationMetric]
    )
    return model

create_model().summary()

In [ ]:
def seed_everything(seed=GCF.SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
#print("X.shape", X.shape)
#print("X[is_train, :].shape", X[is_train, :].shape)
#print("np.expand_dims(X[is_train, :], axis=1).shape", np.expand_dims(X[is_train, :], axis=1).shape)

In [ ]:
seed_everything()

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_correlationMetric',
    patience=GCF.EARLY_STOPPING_PATIENCE,
    min_delta=GCF.EARLY_STOPPING_MIN_DELTA,
    restore_best_weights=True,
)
reduce_lr = ReduceLROnPlateau(
                    monitor='val_correlationMetric',
                    factor=0.5,
                    patience=3,
                    min_lr=1e-5,
                    verbose=1
)

In [ ]:
history = {}

for fold, (is_train, is_test) in enumerate(gkf.split(X, y, time_id)):
    
    print("\n")
    print("-----  fold ------ " + str(fold) + " ------------")
    print("\n")
    
    if fold > 2:
        break
        
    # reinit model
    with strategy.scope():
        model = create_model()
        
        train_x = np.expand_dims(X[is_train, :], axis=1)
        train_y = y[is_train]
        val_x = np.expand_dims(X[is_test, :], axis=1)
        val_y = y[is_test]
        
    # fold 5
    #"""
        history[f'fold-{fold}'] = model.fit(
            train_x, train_y,
            validation_data=(val_x, val_y),
            batch_size=GCF.BATCH_SIZE,
            epochs=GCF.N_EPOCHS,
            callbacks=[early_stopping, reduce_lr],
        )

        model.save(f"ump_transformer_holdout-fold-{fold}.h5")
    #"""

    del model, train_x, train_y, val_x, val_y


In [ ]:
for fold in range(n_splits):    
    print("\n")
    print("-----  fold ------ " + str(fold) + " ------------")
    print("\n")
    
    if fold > 2:
        break
        
    _history = history[f'fold-{fold}'].history
    cols = [h.replace("val_", "") for h in _history.keys() if 'val' in h]
    for c in cols:
        pd.DataFrame(_history)[[c, "val_"+c]].plot()
        plt.title(c)
        #plt.show()

    pd.DataFrame(_history)['lr'].plot()
    plt.title('lr')
    plt.show()
    
    run_epoch = len(_history['loss'])
    best_epoch = run_epoch - GCF.EARLY_STOPPING_PATIENCE
    print(f"best epoch is {best_epoch}")

In [ ]:
for fold, (is_train, is_test) in enumerate(gkf.split(X, y, time_id)):
    print("\n")
    print("-----  fold ------ " + str(fold) + " ------------")
    print("\n")
    
    if fold > 2:
        break
        
    model = tf.keras.models.load_model(f'ump_transformer_holdout-fold-{fold}.h5', 
                                    compile=False,
                                    custom_objects={
                                       "MultiHeadSelfAttention": MultiHeadSelfAttention,
                                       "TransformerBlock": TransformerBlock,
                                    })
    valid_pred = model.predict(np.expand_dims(X[is_test, :], 1))
    valid_pred = valid_pred[:, 0]

    rmse = mean_squared_error(y[is_test], valid_pred, squared=False)
    score = comp_metric(time_id[is_test], y[is_test], valid_pred)
    print(f'RMSE={rmse}, SCORE={score}')    

In [ ]:
# train with all data
"""
seed_everything()

with strategy.scope():
    model = create_model()

reduce_lr = ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.5,
                    patience=3,
                    min_lr=1e-5,
                    verbose=1
)

history = model.fit(
    np.expand_dims(X[is_train + is_test, :], axis=1), y[is_train + is_test],
    batch_size=GCF.BATCH_SIZE,
    epochs=best_epoch + GCF.ALL_TRAIN_ADD_EPOCH,
    callbacks=[reduce_lr],
)

model.save(f"ump_transformer_all_train.h5")
"""

In [ ]:
"""
cols = [h.replace("val_", "") for h in history.history.keys() if 'val' in h]
for c in cols:
    pd.DataFrame(history.history)[[c, "val_"+c]].plot()
    plt.title(c)
    plt.show()
    
pd.DataFrame(history.history)['lr'].plot()
plt.title('lr')
plt.show()
"""